In [1]:
import xgboost as xgb
import catboost as catb
import lightgbm as lgb

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
%matplotlib notebook
import matplotlib.pyplot as plt

In [3]:
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test.csv')

In [4]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [5]:
#validation_df = train_df.sample(frac=0.7, random_state=2011, axis)

In [6]:
X = train_df.drop(['ID_code', 'target'], axis=1)
y = train_df['target']
#X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
X_test = test_df.drop(['ID_code'], axis=1)
X_test_IDs = test_df['ID_code']

## Naive bayes

In [10]:
from sklearn.preprocessing import QuantileTransformer

transformed = pd.DataFrame(QuantileTransformer(output_distribution='normal').fit_transform(X))

In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB

pipeline = make_pipeline(QuantileTransformer(output_distribution='normal'), GaussianNB())
pipeline.fit(X, y)

Pipeline(memory=None,
     steps=[('quantiletransformer', QuantileTransformer(copy=True, ignore_implicit_zeros=False, n_quantiles=1000,
          output_distribution='normal', random_state=None,
          subsample=100000)), ('gaussiannb', GaussianNB(priors=None, var_smoothing=1e-09))])

In [18]:
predictions = pipeline.predict_proba(X_test)[:,1]

### LightGBM

In [28]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1992)

In [29]:
oof_predictions = np.zeros(y.shape)
test_predictions = np.zeros(X_test_IDs.shape)

In [30]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.05,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 2019}


for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y.values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(X.iloc[trn_idx,:], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[val_idx,:], label=y.iloc[val_idx])

    num_round = 20000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=300, early_stopping_rounds=200)
    oof_predictions[val_idx] = clf.predict(X.iloc[val_idx,:], num_iteration=clf.best_iteration)

    
    test_predictions += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

fold 0
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.945314	valid_1's auc: 0.879942
[600]	training's auc: 0.971511	valid_1's auc: 0.889909
[900]	training's auc: 0.986266	valid_1's auc: 0.891142
Early stopping, best iteration is:
[853]	training's auc: 0.984446	valid_1's auc: 0.891278
fold 1
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.944705	valid_1's auc: 0.88011
[600]	training's auc: 0.97126	valid_1's auc: 0.891608
[900]	training's auc: 0.986198	valid_1's auc: 0.893158
[1200]	training's auc: 0.994128	valid_1's auc: 0.893331
Early stopping, best iteration is:
[1265]	training's auc: 0.995099	valid_1's auc: 0.893422
fold 2
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.945523	valid_1's auc: 0.883247
[600]	training's auc: 0.971904	valid_1's auc: 0.892565
[900]	training's auc: 0.986607	valid_1's auc: 0.894309
Early stopping, best iteration is:
[937]	training's auc: 0.9

In [36]:
roc_auc_score(oof_predictions, y)

ValueError: continuous format is not supported

In [31]:
predictions = test_predictions.copy()

In [50]:
predictions = predictions.clip(0, 1).round()

In [51]:
predictions.max()

1.0

In [ ]:
rfc = RandomForestClassifier(criterion='entropy', n_estimators=100, max_depth=7, n_jobs=-1, random_state=123, verbose=1, class_weight={1: .9, 0: .1})
rfc.fit(X, y)

In [ ]:
predictions = rfc.predict(X_valid)

In [ ]:
roc_auc_score(y_valid, predictions)

In [ ]:
from sklearn.preprocessing import StandardScaler, quantile_transform

In [ ]:
qt = quantile_transform(X)

In [ ]:
rfc = RandomForestClassifier(n_estimators=200, max_depth=7, n_jobs=-1, random_state=123, verbose=1, class_weight={1: .9, 0: .1})
rfc.fit(X, y)

In [ ]:
predictions = rfc.predict(X_test)

In [ ]:
roc_auc_score(y_valid, predictions)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
predictions = lr.predict(X_valid)

In [ ]:
roc_auc_score(y_valid, predictions)

In [ ]:
np.mean(y_valid==predictions)

In [ ]:
len(X)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params_dist = {
    'n_estimators':[10, 30, 50, 75, 100, 120],
    'class_weight':[{1: 0.9, 0: 0.1}, {1: 0.99, 0: 0.01}, {1: 0.8, 0: 0.2}, {1: 0.95, 0: 0.05}],
    'max_depth':[2, 3, 5, 7, 9, 11, 13, 15, 20],
}

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1, verbose=1)
randomized_grid_search = RandomizedSearchCV(rfc, param_distributions=params_dist, n_iter=20,
                                            scoring='precision', n_jobs=-1)

In [ ]:
randomized_grid_search.fit(X,y)

In [ ]:
predictions = randomized_grid_search.predict(X_test)

In [ ]:
standard_scaler = StandardScaler()
quantile_scaler = quantile_transform()

In [ ]:
X_scaled = standard_scaler.fit_transform(X)

In [ ]:
X_test_scaled = standard_scaler.transform(X_test)

In [ ]:
log_reg = LogisticRegression()

In [ ]:
log_reg.fit(X_scaled, y)

In [ ]:
predictions = log_reg.predict(X_test_scaled)

In [32]:
submission_df = pd.DataFrame()

In [33]:
submission_df['ID_code'] = X_test_IDs
submission_df['target'] = predictions

In [34]:
submission_df.to_csv('submission_lgbm.csv', index=False)

In [ ]:
randomized_grid_search.best_score_

In [37]:
predictions

array([0.07944363, 0.22120485, 0.19718835, ..., 0.00308006, 0.06966508,
       0.05290362])